In [12]:
# %load ../shared.py
import os
import zipfile as zip
import time
import pandas as pd

#LINUX or PC
OS = "LINUX"

def setConfig(PROJECT_NAME):
    #BASE_PATH = os.popen("cd").read()[:-1]
    # For Mac:
    BASE_PATH = os.popen("pwd").read()[:-1]
    PROJECT_PATH = os.path.join(BASE_PATH, PROJECT_NAME)
    DATASET_PATH = os.path.join(PROJECT_PATH, "dataset")
    DOWNLOAD_PATH = os.path.join(DATASET_PATH, "zip")

    return BASE_PATH, PROJECT_PATH, DATASET_PATH, DOWNLOAD_PATH

#Download and unzip Kaggle dataset
def getDataSet(KAGGLE_COMPETITION, PROJECT_NAME):
    BASE_PATH, PROJECT_PATH, DATASET_PATH, DOWNLOAD_PATH = setConfig(PROJECT_NAME)

    KAGGLE_COMMAND = "kaggle competitions download -c " + KAGGLE_COMPETITION + " -p " + DOWNLOAD_PATH

    try:
        os.mkdir(DOWNLOAD_PATH)
    except OSError:
        print("Directory already exists")
        print("Deleting dataset files..." + DATASET_PATH)
        if (OS == "PC"):
            os.system("del /Q " + DATASET_PATH)
        if (OS == "LINUX"):
            os.system("rm -rf " + DATASET_PATH)
        else:
            print("no OS selected")
            exit(-1)

    print("Download dataset...")
    os.system(KAGGLE_COMMAND)

    print("Unpack dataset...")
    os.system("unzip " + DOWNLOAD_PATH + "/*.zip" + " -d " + DATASET_PATH)
    #unzip = zip.ZipFile(DOWNLOAD_PATH + "\\" + KAGGLE_COMPETITION + ".zip")
    #unzip.extractall(path=DATASET_PATH)
    #unzip.close()
    print("Delete zip file... -- " + PROJECT_PATH + "/zip")
    if (OS == "PC"):
        os.system("del /Q " + PROJECT_PATH + "\\zip")
    if (OS == "LINUX"):
        os.system("rm -rf " + PROJECT_PATH + "/zip")
    else:
        print("no OS selected")
        exit(-1)

#Load Dataset
#Returns test and train Dataframes
def loadDataSet(PROJECT_NAME, indexName="Id"):
    BASE_PATH, PROJECT_PATH, DATASET_PATH, DOWNLOAD_PATH = setConfig(PROJECT_NAME)

   #if (not os.path.exists(DATASET_PATH)):
   #     getDataSet(KAGGLE_COMPETITION,   PROJECT_NAME)

    train = pd.read_csv( os.path.join(DATASET_PATH, "train.csv"), index_col=indexName)
    test =  pd.read_csv( os.path.join(DATASET_PATH, "test.csv"), index_col=indexName)

    return train, test

#Write Kaggle Prediction File
#output ID and SalesPrice
def writeKagglePrediction(PROJECT_NAME, prefix, predict):
    BASE_PATH, PROJECT_PATH, DATASET_PATH, DOWNLOAD_PATH = setConfig(PROJECT_NAME)

    PREDICT_PATH = PROJECT_PATH + "/predict"
    if (not os.path.exists(PREDICT_PATH)):
        os.mkdir(PREDICT_PATH)

    pd.DataFrame(predict).to_csv(PREDICT_PATH+"/"+prefix+"-"+time.strftime("%Y%m%d%H%M%S", time.localtime())+".csv", index=False)



def findBlankColumns(dataset):
    # Check which columns have blank values
    assert  isinstance(dataset, pd.DataFrame)

    for col in list(dataset):
        missing = dataset[col].isna().sum()
        if missing > 1:
            print(col, ": ", dataset[col].isna().sum()," blank values found (", "%.4f pct" % (missing/len(dataset[col])*100),
                  ")")



In [14]:
#Kaggle Constants
KAGGLE_COMPETITION = "titanic"
PROJECT_NAME = "titanic"

getDataSet(KAGGLE_COMPETITION, PROJECT_NAME)

train_raw, test_raw = loadDataSet(PROJECT_NAME, indexName="PassengerId")

train_set = train_raw.copy()

Directory already exists
Deleting dataset files.../home/zach/PycharmProjects/kggle/titanic/titanic/dataset
Download dataset...

Unpack dataset...


100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 288kB/s]


Archive:  /home/zach/PycharmProjects/kggle/titanic/titanic/dataset/zip/titanic.zip
  inflating: /home/zach/PycharmProjects/kggle/titanic/titanic/dataset/gender_submission.csv  
  inflating: /home/zach/PycharmProjects/kggle/titanic/titanic/dataset/test.csv  
  inflating: /home/zach/PycharmProjects/kggle/titanic/titanic/dataset/train.csv  
Delete zip file... -- /home/zach/PycharmProjects/kggle/titanic/titanic/zip


In [16]:
#split train into a train and test set


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#Correlation of Survivors and